# Flora Data Cleaning - January 2026 Extract

Cleans `flora_data_01_2026.csv` which has two formats:
1. **JSON wrapped**: `{"messages": [...]}` with multi-layer escaping
2. **Plain text**: Direct message content

**Input:** `flora_data_01_2026.csv` (356 rows, ~13.8MB)  
**Output:** `flora_data_01_2026_cleaned.csv`

In [ ]:
import pandas as pd
import json
import re
from pathlib import Path

pd.set_option('display.max_colwidth', 100)

DATA_DIR = Path('../data')
RAW_FILE = DATA_DIR / 'flora_data_01_2026.csv'
CLEANED_FILE = DATA_DIR / 'flora_data_01_2026_cleaned.csv'

## 1. Load Raw Data

In [ ]:
df_raw = pd.read_csv(RAW_FILE, encoding='utf-8-sig')
print(f"Loaded {len(df_raw)} rows")
print(f"Columns: {list(df_raw.columns)}")
print(f"Raw file size: {RAW_FILE.stat().st_size / 1024 / 1024:.2f} MB")

## 2. Parsing Functions

Handles two data formats and multi-layer JSON escaping:
- `\"` (1 backslash + quote) = JSON structure quotes
- `\\\"` (3 backslashes + quote) = nested content quotes

In [ ]:
def strip_outer_quotes(val):
    """Remove outer quotes from CSV fields."""
    if not isinstance(val, str):
        return val
    val = val.strip()
    if val.startswith('"') and val.endswith('"') and len(val) > 2:
        return val[1:-1]
    return val


def parse_input_field(val):
    """Parse input - handles JSON and plain text formats."""
    if pd.isna(val) or not isinstance(val, str):
        return {'prompt': None, 'format': 'empty'}
    
    stripped = strip_outer_quotes(val)
    
    if stripped.startswith('{'):
        # JSON format - handle multi-layer escaping
        unescaped = stripped.replace('\\\\\"', '___NESTED___')  # 3 backslashes + quote
        unescaped = unescaped.replace('\\"', '"')                 # 1 backslash + quote  
        unescaped = unescaped.replace('___NESTED___', '\\"')      # restore nested
        
        try:
            data = json.loads(unescaped)
            if 'messages' in data:
                for msg in data['messages']:
                    if msg.get('type') == 'human' or msg.get('role') == 'user':
                        content = msg.get('content', '')
                        if isinstance(content, list):
                            content = ' '.join([c.get('text', '') for c in content if isinstance(c, dict)])
                        return {'prompt': content, 'format': 'json'}
            return {'prompt': None, 'format': 'json_no_human'}
        except json.JSONDecodeError:
            return {'prompt': stripped, 'format': 'json_failed'}
    else:
        return {'prompt': stripped, 'format': 'plain'}


def parse_output_field(val):
    """Parse output - extracts response, model, and token metadata."""
    result = {
        'response': None, 'model': None, 'model_provider': None,
        'input_tokens': None, 'output_tokens': None, 'total_tokens': None,
        'stop_reason': None, 'format': 'empty'
    }
    
    if pd.isna(val) or not isinstance(val, str):
        return result
    
    stripped = strip_outer_quotes(val)
    
    if stripped.startswith('{'):
        unescaped = stripped.replace('\\\\\"', '___NESTED___')
        unescaped = unescaped.replace('\\"', '"')
        unescaped = unescaped.replace('___NESTED___', '\\"')
        
        try:
            data = json.loads(unescaped)
            result['format'] = 'json'
            
            if 'messages' in data:
                for msg in reversed(data['messages']):
                    if msg.get('type') == 'ai':
                        content = msg.get('content', '')
                        if isinstance(content, list):
                            content = ' '.join([c.get('text', '') for c in content if isinstance(c, dict)])
                        result['response'] = content
                        
                        resp_meta = msg.get('response_metadata', {})
                        result['model'] = resp_meta.get('model_name')
                        result['model_provider'] = resp_meta.get('model_provider')
                        result['stop_reason'] = resp_meta.get('stop_reason')
                        
                        usage = msg.get('usage_metadata', {})
                        result['input_tokens'] = usage.get('input_tokens')
                        result['output_tokens'] = usage.get('output_tokens')
                        result['total_tokens'] = usage.get('total_tokens')
                        break
            return result
        except json.JSONDecodeError:
            result['response'] = stripped
            result['format'] = 'json_failed'
            return result
    else:
        result['response'] = stripped
        result['format'] = 'plain'
        return result

## 3. Extract Content

In [ ]:
# Process input fields
print("Processing input fields...")
input_results = df_raw['input'].apply(parse_input_field)
df_raw['prompt'] = input_results.apply(lambda x: x['prompt'])
df_raw['input_format'] = input_results.apply(lambda x: x['format'])

# Process output fields  
print("Processing output fields...")
output_results = df_raw['output'].apply(parse_output_field)
df_raw['response'] = output_results.apply(lambda x: x['response'])
df_raw['model'] = output_results.apply(lambda x: x['model'])
df_raw['model_provider'] = output_results.apply(lambda x: x['model_provider'])
df_raw['input_tokens'] = output_results.apply(lambda x: x['input_tokens'])
df_raw['output_tokens'] = output_results.apply(lambda x: x['output_tokens'])
df_raw['total_tokens'] = output_results.apply(lambda x: x['total_tokens'])
df_raw['stop_reason'] = output_results.apply(lambda x: x['stop_reason'])
df_raw['output_format'] = output_results.apply(lambda x: x['format'])

print(f"\nExtraction Results:")
print(f"  Prompts: {df_raw['prompt'].notna().sum()}/{len(df_raw)}")
print(f"  Responses: {df_raw['response'].notna().sum()}/{len(df_raw)}")
print(f"  Models: {df_raw['model'].notna().sum()}/{len(df_raw)}")
print(f"\nInput formats: {df_raw['input_format'].value_counts().to_dict()}")
print(f"Output formats: {df_raw['output_format'].value_counts().to_dict()}")

## 4. Clean String Columns & Parse Timestamps

In [ ]:
# Clean string columns
string_cols = ['id', 'timestamp', 'name', 'userId', 'sessionId', 'environment']
for col in string_cols:
    if col in df_raw.columns:
        df_raw[col] = df_raw[col].apply(strip_outer_quotes)

# Parse timestamp
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])

# Date features
df_raw['date'] = df_raw['timestamp'].dt.date
df_raw['hour'] = df_raw['timestamp'].dt.hour
df_raw['day_of_week'] = df_raw['timestamp'].dt.day_name()
df_raw['week'] = df_raw['timestamp'].dt.isocalendar().week

print(f"Date range: {df_raw['date'].min()} to {df_raw['date'].max()}")

## 5. Feature Engineering

In [ ]:
# Text length features
df_raw['prompt_length'] = df_raw['prompt'].fillna('').str.len()
df_raw['response_length'] = df_raw['response'].fillna('').str.len()
df_raw['prompt_words'] = df_raw['prompt'].fillna('').str.split().str.len()
df_raw['response_words'] = df_raw['response'].fillna('').str.split().str.len()

# Detect embedded data (JSON in prompts)
def has_embedded_data(text):
    if pd.isna(text):
        return False
    return bool(re.search(r'[{\[].*[}\]]', text, re.DOTALL))

df_raw['has_embedded_data'] = df_raw['prompt'].apply(has_embedded_data)

# Session sequence
df_raw = df_raw.sort_values(['sessionId', 'timestamp'])
df_raw['session_msg_num'] = df_raw.groupby('sessionId').cumcount() + 1
df_raw['is_first_in_session'] = df_raw['session_msg_num'] == 1

# Simplify model names
def simplify_model_name(model):
    if pd.isna(model):
        return 'unknown'
    model = str(model).lower()
    if 'opus' in model: return 'claude-opus'
    elif 'sonnet' in model: return 'claude-sonnet'
    elif 'haiku' in model: return 'claude-haiku'
    elif 'gpt-4.5' in model: return 'gpt-4.5'
    elif 'gpt-4o' in model: return 'gpt-4o'
    elif 'gpt-4o-mini' in model: return 'gpt-4o-mini'
    elif 'gpt-4o-2024-08-06' in model: return 'gpt-4o-08'
    elif 'gpt-4' in model: return 'gpt-4'
    elif 'gpt-3' in model: return 'gpt-3.5'
    return model

df_raw['model_simple'] = df_raw['model'].apply(simplify_model_name)

print(f"Embedded data in prompts: {df_raw['has_embedded_data'].sum()} ({df_raw['has_embedded_data'].mean()*100:.1f}%)")
print(f"Unique sessions: {df_raw['sessionId'].nunique()}")
print(f"\nModel distribution:")
print(df_raw['model_simple'].value_counts())

## 6. Create Clean DataFrame

In [ ]:
clean_columns = [
    'id', 'timestamp', 'date', 'hour', 'day_of_week', 'week',
    'userId', 'sessionId', 'session_msg_num', 'is_first_in_session',
    'prompt', 'response', 'has_embedded_data',
    'prompt_length', 'response_length', 'prompt_words', 'response_words',
    'model', 'model_simple', 'model_provider',
    'input_tokens', 'output_tokens', 'total_tokens', 'stop_reason',
    'input_format', 'output_format',
    'name', 'environment'
]

df_clean = df_raw[clean_columns].copy()
df_clean = df_clean.sort_values('timestamp').reset_index(drop=True)

print(f"Clean dataset: {len(df_clean)} rows, {len(df_clean.columns)} columns")

## 7. Data Quality Report

In [ ]:
print("=" * 60)
print("DATA QUALITY REPORT")
print("=" * 60)

print(f"\nOVERVIEW")
print(f"  Total records: {len(df_clean)}")
print(f"  Date range: {df_clean['date'].min()} to {df_clean['date'].max()}")
print(f"  Unique users: {df_clean['userId'].nunique()}")
print(f"  Unique sessions: {df_clean['sessionId'].nunique()}")

print(f"\nEXTRACTION SUCCESS")
print(f"  Prompts: {df_clean['prompt'].notna().sum()}/{len(df_clean)} ({df_clean['prompt'].notna().mean()*100:.1f}%)")
print(f"  Responses: {df_clean['response'].notna().sum()}/{len(df_clean)} ({df_clean['response'].notna().mean()*100:.1f}%)")
print(f"  Models: {df_clean['model'].notna().sum()}/{len(df_clean)} ({df_clean['model'].notna().mean()*100:.1f}%)")
print(f"  Tokens: {df_clean['total_tokens'].notna().sum()}/{len(df_clean)} ({df_clean['total_tokens'].notna().mean()*100:.1f}%)")

print(f"\nTOKEN USAGE (where available)")
print(f"  Total tokens: {df_clean['total_tokens'].sum():,.0f}")
print(f"  Avg per request: {df_clean['total_tokens'].mean():,.0f}")

print(f"\nTOP 5 USERS")
user_counts = df_clean['userId'].value_counts().head(5)
cumsum = 0
for i, (user, count) in enumerate(user_counts.items(), 1):
    cumsum += count
    print(f"  User {i}: {count} msgs ({count/len(df_clean)*100:.1f}%) - cumulative: {cumsum/len(df_clean)*100:.1f}%")

## 8. Save Cleaned Data

In [ ]:
df_clean.to_csv(CLEANED_FILE, index=False)

print(f"Saved: {CLEANED_FILE}")
print(f"Size: {CLEANED_FILE.stat().st_size / 1024:.1f} KB")
print(f"Compression: {RAW_FILE.stat().st_size / CLEANED_FILE.stat().st_size:.1f}x smaller")

## 9. Sample Preview

In [ ]:
print("SAMPLE PROMPTS")
print("=" * 60)
for i, row in df_clean.head(5).iterrows():
    prompt = row['prompt']
    if prompt:
        display = prompt[:200].replace('\n', ' ')
        if len(prompt) > 200:
            display += "..."
        print(f"\n[{row['model_simple']}] {row['input_format']}")
        print(f"{display}")

In [29]:
# Check sample prompts to tune intent categorization
import pandas as pd
from pathlib import Path

DATA_DIR = Path('/Users/nicolodiferdinando/Desktop/School/Semesters/Winter26/IME482/Pareto Analysis/data')
df = pd.read_csv(DATA_DIR / 'flora_data_01_2026_cleaned.csv')

# Sample of prompts
print("Sample prompts (first 150 chars):\n")
for i, prompt in enumerate(df['prompt'].head(20)):
    if prompt:
        print(f"{i}: {prompt[:150]}...")
        print()

Sample prompts (first 150 chars):

0: Analyze the following data and provide a concise summary of the key findings. Keep your response in a short paragraph of a few sentences and impactful...

1: How did the tasks flow through the statuses...

2: Analyze the following data and provide a concise summary of the key findings. Keep your response in a short paragraph of a few sentences and impactful...

3: how are my top 25 initiatives doing for deliverability...

4: Analyze the following data and provide a concise summary of the key findings. Keep your response in a short paragraph of a few sentences and impactful...

5: Show me all of my off-track initiatives....

6: Analyze the following data and provide a concise summary of the key findings. Keep your response in a short paragraph of a few sentences and impactful...

7: Show me all of my off-track initiatives....

8: Analyze data work period 'REBEL-2025 SP26' ...

9: provide a performance analysis for work period 'REBEL-2025 SP26'...

1

In [30]:
import re

def categorize_intent(prompt):
    """Categorize prompt intent based on ACTION + TARGET patterns."""
    if pd.isna(prompt):
        return 'Unknown'
    
    prompt_lower = prompt.lower()
    
    # Executive Reporting
    if re.search(r'(provide|generate|create|give).*(summary|report|executive|overview)', prompt_lower):
        return 'Executive Reporting'
    
    # Data Analysis
    if re.search(r'analyze.*(data|following|metrics|phase|cards)', prompt_lower):
        return 'Data Analysis'
    
    # Sprint Retrospective (including typo "reto")
    if re.search(r'sprint.*(retro|reto|review|analysis|recap)', prompt_lower):
        return 'Sprint Retrospective'
    
    # Sprint Planning
    if re.search(r'sprint.*(plan|prep|upcoming|next)', prompt_lower):
        return 'Sprint Planning'
    
    # Metrics Query
    if re.search(r'(velocity|throughput|cycle time|lead time|burndown)', prompt_lower):
        return 'Metrics Query'
    
    # Status Update
    if re.search(r'(status|progress|update|where.*stand)', prompt_lower):
        return 'Status Update'
    
    # Information Request
    if re.search(r'^(tell me|what is|what are|explain|describe|how)', prompt_lower):
        return 'Information Request'
    
    # Initiative/Project Query
    if re.search(r'(initiative|project|epic|feature)', prompt_lower):
        return 'Initiative Query'
    
    # Team Query
    if re.search(r'(team|squad|group)', prompt_lower):
        return 'Team Query'
    
    return 'Other'

# Apply and check distribution
df['intent'] = df['prompt'].apply(categorize_intent)
print("Current intent distribution:")
print(df['intent'].value_counts())
print(f"\nTotal: {len(df)}")

Current intent distribution:
intent
Executive Reporting     118
Other                    85
Metrics Query            57
Information Request      46
Data Analysis            17
Status Update            10
Team Query                9
Initiative Query          8
Sprint Retrospective      6
Name: count, dtype: int64

Total: 356


In [31]:
# Check "Other" prompts to see if we can categorize more
other_prompts = df[df['intent'] == 'Other']['prompt'].head(30)
print("'Other' category prompts:\n")
for i, prompt in enumerate(other_prompts):
    print(f"{i}: {prompt[:120]}...")
    print()

'Other' category prompts:

0: provide a performance analysis for work period 'REBEL-2025 SP26'...

1: why is my portfolio performance decreasing in december...

2: `...

3: can you compare december with november data\n...

4: Analyze [@REBEL-Scrum] from 8/1/25 through 12/31/25 and identify where the main bottlenecks, and which workflow states c...

5: ""...

6: show be results across my portfolio for SP26...

7: ""...

8: Can you show Readiness over this sprint...

9: what about sprint readiness...

10: For @work_period:ef237015-9250-4b1a-9797-6b983f46482b[REBEL-2025 SP26]@parent:board:b1192a4c-f0fb-4fac-aa9a-17b284d95b8e...

11: define stale task...

12: are there any cards in @work_period:f19de16c-bd76-4113-93ef-cf4fb8930b56[FBI-2026 SP02]@parent:board:c18efa8d-d631-4ef2-...

13: Create a chart that shows the Commitment Reliability by Story Type for these sprints @work_period:f19de16c-bd76-4113-93e...

14: Retrospective Insights Trend for @work_period:ea1b49f8-77ab-4180-869b-e9702a2e